# Heterogeneous mixing

By default, the Summer compartmental model assumes that each person in the model comes into contact with every other person at the same rate (homogeneous mixing). This assumption is preserved even after stratification has been applied (by default), such that the force of infection calculation will be the same for all model strata implemented. This notebook considers the incorporation of heterogeneous mixing patterns. For example, children may tend to interact more with other children than they do with elderly people (this is termed with-like or assortative mixing). When considering the impact of control strategies, especially strategies targeting sub groups of people, it is likely to be important to consider these differences in contact patterns.

## Standard preliminaries
Before we get into the code for heterogeneous mixing, let's start off with some of our standard (or "boilerplate") code to get everything set up.

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import plotly.express as px

from summer import CompartmentalModel

pd.options.plotting.backend = "plotly"

In [ ]:
# The data import module lives in a file on AuTuMN github - download it for colab use
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/philippines/import_phl_data.py

import import_phl_data

In [ ]:
from import_phl_data import get_population_and_epi_data

analysis_start_date = datetime(2021,1,1)  # Define the start date
analysis_end_date = analysis_start_date + timedelta(days=300)  # Define the duration

# Shareable google drive links
PHL_DOH_LINK = "1fFKoNVan7PS6BpBr01nwByNTLLu6z1AA"  # sheet 05 daily report.
PHL_FASSSTER_LINK = "15eDyTjXng2Zh38DVhmeNy0nQSqOMlGj3" # Fassster google drive zip file.
initial_population, df = get_population_and_epi_data(PHL_DOH_LINK, PHL_FASSSTER_LINK) 
notifications_target = df[analysis_start_date: analysis_end_date]["cases"]  # Could be used as calibration target later

# We define a day zero for the analysis.
COVID_BASE_DATE = datetime(2019, 12, 31)

# Integer representation of the start and end dates.
start_date_int = (analysis_start_date - COVID_BASE_DATE).days
end_date_int = (analysis_end_date - COVID_BASE_DATE).days

In [ ]:
initial_population

## Getting mixing matrices

In [ ]:
simple_matrix = np.array([[1., 0.5], [0.5, 1.]])
print(simple_matrix.shape)
px.imshow(simple_matrix)

Get a mixing matrix that we can use for Malaysia.
This is not actually empiric data for Malaysia, but is what we have been using
and has the right format.

In [ ]:
# Get the mixing matrix for Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

In [ ]:
# Let's have a look at the matrix using plotly express
print(age_mixing_matrix.shape)
px.imshow(age_mixing_matrix)